In [1]:
import pandas as pd
import chardet
import time
t1=time.time()
#1、读入数据
f = pd.read_csv('HW1data.csv', encoding='ISO-8859-1')
# 查看编码
# with open('HW1data.csv','rb') as data:
#     res=chardet.detect(data.read(1000))
# print(res)
#2、补全数据
genderdict = {}
agedict = {}
Neighdict = {}
totallen = f.shape[0]
print(totallen)
# 记录age,gender,Neighbourhood信息
for i in range(0, totallen):
    if len(str(f['PatientId'][i])) > 3:
        if str(f['Gender'][i]) != 'nan':
            genderdict[str(f['PatientId'][i])[:-2]] = f['Gender'][i]
        if str(f['Age'][i]) != 'nan':
            agedict[str(f['PatientId'][i])[:-2]] = f['Age'][i]
        if str(f['Neighbourhood'][i]) != 'nan':
            Neighdict[str(f['PatientId'][i])[:-2]] = f['Neighbourhood'][i]
print(f.shape)



667536
(667536, 14)


In [2]:
# print(len(agedict.keys()))
# print(len(genderdict.keys()))
# print(len(Neighdict.keys()))
#遍历补全信息
for i in range(0,totallen):
    if len(str(f['PatientId'][i])) > 3:
        if str(f['Gender'][i]) == 'nan':
            f['Gender'][i] = genderdict[str(f['PatientId'][i])[:-2]]
        if str(f['Age'][i]) == 'nan':
            f['Age'][i] = agedict[str(f['PatientId'][i])[:-2]]
        if str(f['Neighbourhood'][i]) == 'nan':
            f['Neighbourhood'][i] = Neighdict[str(f['PatientId'][i])[:-2]]
#删除空行
f.dropna(axis=0, subset=['PatientId', 'ScheduledDay', 'AppointmentDay', 'SMS_received', 'No-show'], how='any',
         inplace=True)
print(f.shape)
totallen = f.shape[0]

print(f.isnull().sum())


d:\anaconda3\envs\env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\anaconda3\envs\env\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
d:\anaconda3\envs\env\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


(413654, 14)
PatientId             0
AppointmentID     34490
Gender                0
ScheduledDay          0
AppointmentDay        0
Age                   0
Neighbourhood         0
Scholarship       35013
Hipertension      34734
Diabetes          34652
Alcoholism        34956
Handcap           34882
SMS_received          0
No-show               0
dtype: int64


In [3]:
#4、补0
f['Scholarship']=f['Scholarship'].fillna(0)
f['Hipertension']=f['Hipertension'].fillna(0)
f['Diabetes']=f['Diabetes'].fillna(0)
f['Handcap']=f['Handcap'].fillna(0)
f['Alcoholism']=f['Alcoholism'].fillna(0)
print(f.isnull().sum())



PatientId             0
AppointmentID     34490
Gender                0
ScheduledDay          0
AppointmentDay        0
Age                   0
Neighbourhood         0
Scholarship           0
Hipertension          0
Diabetes              0
Alcoholism            0
Handcap               0
SMS_received          0
No-show               0
dtype: int64


In [4]:
#5、删除 id列
f=f.drop(columns = ['PatientId'])
f=f.drop(columns = ['AppointmentID'])
print(f.shape)


(413654, 12)


In [5]:
# print(f['Age'].min())
# print(f['Age'].max())
#6、删除age为-1的行
f=f[~f['Age'].isin([-1])]

print(f['Age'].min())
print(f['Age'].max())
print(f.shape)

0.0
115.0
(413650, 12)


In [6]:
#7、创建新列
new_df = pd.DataFrame((pd.to_datetime(f['AppointmentDay']) - pd.to_datetime(f['ScheduledDay'])).dt.total_seconds()/60/60/24)
f['Delta_Day']=list(new_df[0])

f=f[f['Delta_Day']>=0]# 删除负数天数的行
f['Delta_Day']=f['Delta_Day'].map(lambda x:int(x))
print(f.shape)


(71959, 13)


In [7]:
#8、计算星期
f['AppointmentDay']=pd.to_datetime(f['AppointmentDay'])
#输出这一天是周中的第几天，Monday, Sunday
f['ADay_DOW']=f['AppointmentDay'].dt.day_name()

f['ScheduledDay']=pd.to_datetime(f['ScheduledDay'])
#输出这一天是周中的第几天，Monday, Sunday
f['SDay_DOW']=f['ScheduledDay'].dt.day_name()
print(f.shape)


(71959, 15)


In [8]:
#9、删除原有的两列Day
f=f.drop(columns = ['ScheduledDay'])
f=f.drop(columns = ['AppointmentDay'])
t2=time.time()
print(f.head())
# print(t2-t1)
print(f.shape)

   Gender   Age        Neighbourhood  Scholarship  Hipertension  Diabetes  \
0       F  21.0  ILHA DE SANTA MARIA          0.0           0.0       0.0   
2       F  41.0              ITARARÉ          0.0           0.0       0.0   
12      M  82.0      JARDIM DA PENHA          0.0           0.0       0.0   
13      F  61.0       BENTO FERREIRA          0.0           0.0       0.0   
14      F  63.0       JARDIM CAMBURI          0.0           0.0       0.0   

    Alcoholism  Handcap  SMS_received No-show  Delta_Day   ADay_DOW SDay_DOW  
0          0.0      0.0           1.0      No          7  Wednesday  Tuesday  
2          0.0      0.0           0.0      No         19     Monday  Tuesday  
12         0.0      0.0           1.0      No         27     Monday   Monday  
13         0.0      0.0           1.0      No         13     Friday   Friday  
14         0.0      0.0           0.0     Yes         48    Tuesday  Tuesday  
(71959, 13)


In [9]:
#f.to_csv("Result.csv")# 输出到csv文件

In [10]:
# f.drop_duplicates(inplace=True)
# print(f.shape)